<a href="https://colab.research.google.com/github/e71828/Adaptive-Filter/blob/main/Chapter2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np
from scipy import linalg
np.set_printoptions(precision=5, suppress=True)

In [35]:
Ra = 1/4*np.asarray(linalg.toeplitz([4,3,2,1]))
pa = np.array([1/2, 3/8, 2/8, 1/8])
R = [Ra]
p = [pa]

Rb = np.asarray(linalg.toeplitz([1,0.8,0.64,0.512]))
pb = 1/4*np.array([0.4096, 0.512, 0.64, 0.8])
R.append(Rb)
p.append(pb)

Rc = 1/3*np.asarray(linalg.toeplitz([3, -2, 1]))
pc = np.array([-2,1,-1/2])
R.append(Rc)
p.append(pc)

for Ra,pa in zip(R,p):
    print("{:=^50s}".format("autocorrelation R"))
    print(Ra)
    print("{:-^50s}".format("vector p"))
    print(pa)

================autocorrelation R=================
[[1.   0.75 0.5  0.25]
 [0.75 1.   0.75 0.5 ]
 [0.5  0.75 1.   0.75]
 [0.25 0.5  0.75 1.  ]]
---------------------vector p---------------------
[0.5   0.375 0.25  0.125]
================autocorrelation R=================
[[1.    0.8   0.64  0.512]
 [0.8   1.    0.8   0.64 ]
 [0.64  0.8   1.    0.8  ]
 [0.512 0.64  0.8   1.   ]]
---------------------vector p---------------------
[0.1024 0.128  0.16   0.2   ]
================autocorrelation R=================
[[ 1.      -0.66667  0.33333]
 [-0.66667  1.      -0.66667]
 [ 0.33333 -0.66667  1.     ]]
---------------------vector p---------------------
[-2.   1.  -0.5]


In [36]:
## Wiener solution
w0 = []
for Ra,pa in zip(R,p):
    wa = linalg.inv(Ra)@pa
    w0.append(wa)
print(w0)

[array([0.5, 0. , 0. , 0. ]), array([0. , 0. , 0. , 0.2]), array([-2.4375, -0.75  , -0.1875])]


In [37]:
## lambda_max
lmax = []
for Ra in R:
    D,V = linalg.eig(Ra)
    print(D)
    print(max(D))
    lmax.append(max(D))

[2.77475+0.j 0.85355+0.j 0.22525+0.j 0.14645+0.j]
(2.774754878398196+0j)
[3.10318+0.j 0.55926+0.j 0.20882+0.j 0.12874+0.j]
(3.10318208944141+0j)
[2.12409+0.j 0.66667+0.j 0.20924+0.j]
(2.124093774423005+0j)


In [38]:
def Gradient_Descent(pa, Ra, mu=1/5):
    w0 = np.zeros(pa.shape)
    g = -2*pa + 2*Ra @ w0
    i = 0
    w0_rec = [ ]
    while(linalg.norm(g)> 1e-6):
        w1 = w0 - mu*g
        w0 = w1
        g = -2*pa + 2*Ra @ w0
        #  record the first ten iterations
        if i<10:
            w0_rec.append(w0)
            i = i +1
        else:
            pass
    return w0,w0_rec

In [39]:
from sympy.abc import sigma
for Ra,pa in zip(R,p):
    w0,w0_rec = Gradient_Descent(pa, Ra)
    print("{:=^50s}".format("New condition"))
    print('last: ', w0)
    print("{:-^50s}".format("fist ten iterations"))
    for w in w0_rec:
        print(w)
    print("{:-^50s}".format("MSE"))
    epsilon = sigma**2 - w0 @ pa
    print('epsilon_min: ', epsilon, '\n\n')

==================New condition===================
last:  [ 0.5  0.  -0.   0. ]
---------------fist ten iterations----------------
[0.2  0.15 0.1  0.05]
[0.25 0.14 0.06 0.  ]
[ 0.296  0.141  0.044 -0.021]
[ 0.3286  0.1368  0.0312 -0.0336]
[ 0.35324  0.13086  0.02204 -0.03974]
[ 0.37225  0.12388  0.01524 -0.04195]
[ 0.38733  0.11647  0.01012 -0.04174]
[ 0.39961  0.109    0.00618 -0.04011]
[ 0.40984  0.10168  0.00312 -0.03768]
[ 0.41854  0.09466  0.0007  -0.03487]
-----------------------MSE------------------------
epsilon_min:  sigma**2 - 0.249999933776242 


==================New condition===================
last:  [ 0.  -0.   0.   0.2]
---------------fist ten iterations----------------
[0.04096 0.0512  0.064   0.08   ]
[0.01638 0.02785 0.04993 0.08602]
[0.01148 0.02467 0.05332 0.10515]
[0.00477 0.01835 0.05151 0.11736]
[0.00073 0.01415 0.05026 0.12826]
[-0.00227  0.01054  0.0484   0.1371 ]
[-0.00424  0.00767  0.04637  0.14454]
[-0.00551  0.00532  0.04421  0.15079]
[-0.00625  0.00341  0